# Elevation Map of Arcadia, Greece using Google Earth Engine and Folium

## Imports and Initial Setup

In [9]:
import folium
import osmnx as ox
import ee
# Initialize Earth Engine
ee.Initialize(project='insert GEE project ID here')


# Load the boundary for Arcadia, Greece using OpenStreetMap
place_name = "Arcadia, Greece"
gdf = ox.geocode_to_gdf(place_name)

# Get the bounds (latitude and longitude) of Arcadia to center the map
bounds = gdf.geometry.total_bounds  # [minx, miny, maxx, maxy]
center_lat, center_lon = (bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2  # center of the region


## Clipping and superimposing GEE data

In [10]:
# Create a Folium map centered on Arcadia, with no background map layer
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, control_scale=True, tiles=None)

# Add Arcadia boundary outline to the map (red)
folium.GeoJson(
    gdf.geometry.__geo_interface__,  # Convert the GeoDataFrame to GeoJSON
    style_function=lambda x: {
        'color': 'red',
        'weight': 3,
        'fillOpacity': 0  # No fill, just the outline
    }
).add_to(m)

# Define Arcadia boundary as a geojson object, properly accessing coordinates
arcadia_coords = gdf.geometry.iloc[0].__geo_interface__['coordinates']
arcadia_boundary = ee.Geometry.Polygon(arcadia_coords)

# Load SRTM elevation data (or any geophysical data)
elevation = ee.Image("USGS/SRTMGL1_003")

# Clip the data to the Arcadia boundary
elevation_clipped = elevation.clip(arcadia_boundary)

# Set visualization parameters (e.g., color palette for elevation)
elevation_vis = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Generate a folium tile layer from the clipped SRTM data
url = elevation_clipped.getMapId(elevation_vis)['tile_fetcher'].url_format
folium.TileLayer(url, attr='USGS', name='Elevation').add_to(m)

# Manually set the map bounds to focus on Arcadia
m.fit_bounds([(bounds[1], bounds[0]), (bounds[3], bounds[2])])

# Display the map
m


In [11]:
import folium
import ee
import osmnx as ox

# Initialize Earth Engine
ee.Initialize(project='geo-tut-457110')

# Load the boundary for Arcadia, Greece using OpenStreetMap
place_name = "Arcadia, Greece"
gdf = ox.geocode_to_gdf(place_name)

# Get the bounds (latitude and longitude) of Arcadia to center the map
bounds = gdf.geometry.total_bounds  # [minx, miny, maxx, maxy]
center_lat, center_lon = (bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2  # center of the region

# Create a Folium map centered on Arcadia, with no background map layer
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, control_scale=True, tiles=None)

# Add Arcadia boundary outline to the map (red)
folium.GeoJson(
    gdf.geometry.__geo_interface__,  # Convert the GeoDataFrame to GeoJSON
    style_function=lambda x: {
        'color': 'red',
        'weight': 3,
        'fillOpacity': 0  # No fill, just the outline
    }
).add_to(m)

# Define Arcadia boundary as a geojson object
arcadia_coords = gdf.geometry.iloc[0].__geo_interface__['coordinates']
arcadia_boundary = ee.Geometry.Polygon(arcadia_coords)

# Load SRTM elevation data (or any geophysical data)
elevation = ee.Image("USGS/SRTMGL1_003")

# Clip the data to the Arcadia boundary
elevation_clipped = elevation.clip(arcadia_boundary)

# Set visualization parameters (e.g., color palette for elevation)
elevation_vis = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Generate a folium tile layer from the clipped SRTM data
elevation_url = elevation_clipped.getMapId(elevation_vis)['tile_fetcher'].url_format
elevation_layer = folium.TileLayer(
    tiles=elevation_url, 
    attr='USGS', 
    name='Elevation',
    overlay=True,
    control=True,
    opacity=0.7  # Set initial opacity for transparency
)

# To ensure the layer is only visible within Arcadia, clip the map to the Arcadia region
# Add the elevation layer to the map with bounding clip
elevation_layer.add_to(m)

# Add OpenStreetMap as the background map (you can toggle this off if needed)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap', control=True).add_to(m)

# Add Google Satellite Layer (can toggle on/off) with proper attribution
folium.TileLayer(
    'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', 
    name='Google Satellite',
    attr='Google',  # Attribution for Google Maps
    control=True
).add_to(m)

# Add Roads/streets (via OSM) with attribution
folium.TileLayer(
    'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    name='Roads (OSM)',
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',  # Attribution for OSM
    control=True
).add_to(m)

# Add the layer control to toggle layers on/off
folium.LayerControl().add_to(m)

# Add a legend for elevation layer (example)
elevation_legend = """
<div style="position: absolute; bottom: 50px; left: 50px; background: white; padding: 5px; border-radius: 5px;">
    <b>Elevation (m)</b><br>
    <i style="background-color: blue; padding: 5px;"></i> 0 - 500 m<br>
    <i style="background-color: green; padding: 5px;"></i> 500 - 1000 m<br>
    <i style="background-color: yellow; padding: 5px;"></i> 1000 - 1500 m<br>
    <i style="background-color: orange; padding: 5px;"></i> 1500 - 2000 m<br>
    <i style="background-color: red; padding: 5px;"></i> 2000 - 3000 m
</div>
"""
m.get_root().html.add_child(folium.Element(elevation_legend))

# Display the map
m
